# XYI GST circuit generation and conversion

Leon Maurer

2020-07-24

In [1]:
import cirq
import pygsti
from pygsti.modelpacks import smq1Q_XYI
import numpy as np
import tqdm

## 1. Generate the GST circuits

### Make target gate set $\{\sqrt{X},\sqrt{Y},I\}$

In [2]:
target_model = smq1Q_XYI.target_model()

### Preparation and measurement fiducials, germs

In [3]:
preps = smq1Q_XYI.prep_fiducials()
effects = smq1Q_XYI.meas_fiducials()
germs = smq1Q_XYI.germs()

In [4]:
preps

[Circuit({}@(0)),
 Circuit(Gxpi2:0@(0)),
 Circuit(Gypi2:0@(0)),
 Circuit(Gxpi2:0Gxpi2:0@(0)),
 Circuit(Gxpi2:0Gxpi2:0Gxpi2:0@(0)),
 Circuit(Gypi2:0Gypi2:0Gypi2:0@(0))]

In [5]:
effects

[Circuit({}@(0)),
 Circuit(Gxpi2:0@(0)),
 Circuit(Gypi2:0@(0)),
 Circuit(Gxpi2:0Gxpi2:0@(0)),
 Circuit(Gxpi2:0Gxpi2:0Gxpi2:0@(0)),
 Circuit(Gypi2:0Gypi2:0Gypi2:0@(0))]

In [6]:
germs

[Circuit([]@(0)),
 Circuit(Gxpi2:0@(0)),
 Circuit(Gypi2:0@(0)),
 Circuit(Gxpi2:0Gypi2:0@(0)),
 Circuit(Gxpi2:0Gxpi2:0Gypi2:0@(0))]

### Construct pyGSTi circuits

This takes a few seconds.

In [7]:
max_lengths = list(np.logspace(0, 10, 11, base=2, dtype=int))

In [8]:
print(max_lengths)

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]


In [10]:
circuits = pygsti.construction.create_lsgst_circuits(target_model, preps, effects, germs, max_lengths)
dict(enumerate(circuits))

{0: Circuit({}@(0)),
 1: Circuit(Gxpi2:0@(0)),
 2: Circuit(Gypi2:0@(0)),
 3: Circuit(Gxpi2:0Gxpi2:0@(0)),
 4: Circuit(Gxpi2:0Gxpi2:0Gxpi2:0@(0)),
 5: Circuit(Gypi2:0Gypi2:0Gypi2:0@(0)),
 6: Circuit(Gxpi2:0Gypi2:0@(0)),
 7: Circuit(Gxpi2:0Gxpi2:0Gxpi2:0Gxpi2:0@(0)),
 8: Circuit(Gxpi2:0Gypi2:0Gypi2:0Gypi2:0@(0)),
 9: Circuit(Gypi2:0Gxpi2:0@(0)),
 10: Circuit(Gypi2:0Gypi2:0@(0)),
 11: Circuit(Gypi2:0Gxpi2:0Gxpi2:0@(0)),
 12: Circuit(Gypi2:0Gxpi2:0Gxpi2:0Gxpi2:0@(0)),
 13: Circuit(Gypi2:0Gypi2:0Gypi2:0Gypi2:0@(0)),
 14: Circuit(Gxpi2:0Gxpi2:0Gypi2:0@(0)),
 15: Circuit(Gxpi2:0Gxpi2:0Gxpi2:0Gxpi2:0Gxpi2:0@(0)),
 16: Circuit(Gxpi2:0Gxpi2:0Gypi2:0Gypi2:0Gypi2:0@(0)),
 17: Circuit(Gxpi2:0Gxpi2:0Gxpi2:0Gypi2:0@(0)),
 18: Circuit(Gxpi2:0Gxpi2:0Gxpi2:0Gxpi2:0Gxpi2:0Gxpi2:0@(0)),
 19: Circuit(Gxpi2:0Gxpi2:0Gxpi2:0Gypi2:0Gypi2:0Gypi2:0@(0)),
 20: Circuit(Gypi2:0Gypi2:0Gypi2:0Gxpi2:0@(0)),
 21: Circuit(Gypi2:0Gypi2:0Gypi2:0Gxpi2:0Gxpi2:0@(0)),
 22: Circuit(Gypi2:0Gypi2:0Gypi2:0Gxpi2:0Gxpi2:0Gxpi2:0@(

In [11]:
len(circuits)

1624

In [12]:
circuit = circuits[111]

In [13]:
print(circuit)

Qubit 0 ---|Gxpi2|-|Gxpi2|-| |-| |-|Gxpi2|---



In [15]:
circuit.layer_with_idles(1)

(Label[Gxpi2:0],)

In [16]:
circuit.layer_with_idles(2)

(Label[I:0],)

## 2. Convert to runable `cirq.Circuit`'s

### Setup

Now, we need to map the qubit names from pyGSTi (`0`, `1`, etc.) into cirq qubits. Here, there's only one qubit to convert. I'm using `cirq.GridQubit(8, 3)` because that was used in Kevin's notebook.

In [17]:
q0 = cirq.GridQubit(8, 3)
qubit_label_dict = {0: q0}

### Testing examples

Do an example conversion.

In [18]:
pygsti_circuit = circuits[111]
print('pyGSTi:')
print(pygsti_circuit)
print('Cirq:')
print(pygsti_circuit.convert_to_cirq(qubit_label_dict))

pyGSTi:
Qubit 0 ---|Gxpi2|-|Gxpi2|-| |-| |-|Gxpi2|---

Cirq:
(8, 3): ───X^0.5───X^0.5───────────X^0.5───


Do another example conversion.

In [19]:
pygsti_circuit = circuits[90]
print('pyGSTi:')
print(pygsti_circuit)
print('Cirq:')
print(pygsti_circuit.convert_to_cirq(qubit_label_dict))

pyGSTi:
Qubit 0 ---|Gypi2|-|Gypi2|-|Gypi2|-|Gypi2|-|Gxpi2|-|Gxpi2|-|Gxpi2|---

Cirq:
(8, 3): ───Y^0.5───Y^0.5───Y^0.5───Y^0.5───X^0.5───X^0.5───X^0.5───


Now, lets try the same thing but specifing a wait duration for the idle operation.

In [20]:
wait_duration = cirq.Duration(nanos=100)

In [21]:
pygsti_circuit = circuits[111]
print('pyGSTi:')
print(pygsti_circuit)
print('Cirq:')
print(pygsti_circuit.convert_to_cirq(qubit_label_dict, wait_duration))

pyGSTi:
Qubit 0 ---|Gxpi2|-|Gxpi2|-| |-| |-|Gxpi2|---

Cirq:
(8, 3): ───X^0.5───X^0.5───WaitGate(100 ns)───WaitGate(100 ns)───X^0.5───


In [22]:
pygsti_circuit = circuits[90]
print('pyGSTi:')
print(pygsti_circuit)
print('Cirq:')
print(pygsti_circuit.convert_to_cirq(qubit_label_dict, wait_duration))

pyGSTi:
Qubit 0 ---|Gypi2|-|Gypi2|-|Gypi2|-|Gypi2|-|Gxpi2|-|Gxpi2|-|Gxpi2|---

Cirq:
(8, 3): ───Y^0.5───Y^0.5───Y^0.5───Y^0.5───X^0.5───X^0.5───X^0.5───


### The real thing

Now, do the real conversion.

In [23]:
cirq_circuits = [c.convert_to_cirq(qubit_label_dict, wait_duration) for c in tqdm.tqdm(circuits)]

100%|██████████| 1624/1624 [00:07<00:00, 214.84it/s]


Note that we're missing the measurments, the idle operations don't have a time associated with them, and the first circuit is empty (it's should just be an idle). Otherwise, the results look good, and those things should be easy to fix.

In [24]:
cirq_circuits

[,
 (8, 3): ───X^0.5───,
 (8, 3): ───Y^0.5───,
 (8, 3): ───X^0.5───X^0.5───,
 (8, 3): ───X^0.5───X^0.5───X^0.5───,
 (8, 3): ───Y^0.5───Y^0.5───Y^0.5───,
 (8, 3): ───X^0.5───Y^0.5───,
 (8, 3): ───X^0.5───X^0.5───X^0.5───X^0.5───,
 (8, 3): ───X^0.5───Y^0.5───Y^0.5───Y^0.5───,
 (8, 3): ───Y^0.5───X^0.5───,
 (8, 3): ───Y^0.5───Y^0.5───,
 (8, 3): ───Y^0.5───X^0.5───X^0.5───,
 (8, 3): ───Y^0.5───X^0.5───X^0.5───X^0.5───,
 (8, 3): ───Y^0.5───Y^0.5───Y^0.5───Y^0.5───,
 (8, 3): ───X^0.5───X^0.5───Y^0.5───,
 (8, 3): ───X^0.5───X^0.5───X^0.5───X^0.5───X^0.5───,
 (8, 3): ───X^0.5───X^0.5───Y^0.5───Y^0.5───Y^0.5───,
 (8, 3): ───X^0.5───X^0.5───X^0.5───Y^0.5───,
 (8, 3): ───X^0.5───X^0.5───X^0.5───X^0.5───X^0.5───X^0.5───,
 (8, 3): ───X^0.5───X^0.5───X^0.5───Y^0.5───Y^0.5───Y^0.5───,
 (8, 3): ───Y^0.5───Y^0.5───Y^0.5───X^0.5───,
 (8, 3): ───Y^0.5───Y^0.5───Y^0.5───X^0.5───X^0.5───,
 (8, 3): ───Y^0.5───Y^0.5───Y^0.5───X^0.5───X^0.5───X^0.5───,
 (8, 3): ───Y^0.5───Y^0.5───Y^0.5───Y^0.5───Y^0.5───Y^0.5

## 3. Run the circuits

In [25]:
simulator = cirq.Simulator()

In [26]:
test_circuit = cirq_circuits[90]
print(test_circuit)

(8, 3): ───Y^0.5───Y^0.5───Y^0.5───Y^0.5───X^0.5───X^0.5───X^0.5───


In [27]:
result = simulator.simulate(test_circuit)
print(result)

measurements: (no measurements)
output vector: (0.5-0.5j)|0⟩ + (0.5+0.5j)|1⟩


In [28]:
type(result)

cirq.sim.wave_function_simulator.WaveFunctionTrialResult

In [29]:
import copy
test_circuit_with_measurement = copy.deepcopy(cirq_circuits[90])
test_circuit_with_measurement.append(cirq.measure(q0, key='result'))
samples = simulator.run(test_circuit_with_measurement, repetitions=1000)

In [30]:
print(samples.histogram(key='result'))

Counter({1: 522, 0: 478})


In [31]:
type(samples)

cirq.study.trial_result.TrialResult

In [32]:
samples.histogram(key='result').values()

dict_values([522, 478])

In [33]:
results = []
for circuit in tqdm.tqdm(cirq_circuits):
    circuit_with_measurement = copy.deepcopy(circuit)
    circuit_with_measurement.append(cirq.measure(q0, key='result'))
    samples = simulator.run(circuit_with_measurement, repetitions=1000)
    results.append(samples)

100%|██████████| 1624/1624 [00:36<00:00, 44.36it/s]


In [35]:
dataset = pygsti.objects.dataset.DataSet(outcome_labels=[0,1])
for pygsti_cirquit, trial_result in zip(circuits, results):
    result_dict = trial_result.histogram(key='result')
    dataset.add_count_dict(circuit, result_dict)

TypeError: 'int' object is not iterable

In [36]:
dataset = pygsti.objects.dataset.DataSet(outcome_labels=[0,1])

TypeError: 'int' object is not iterable